<a href="https://colab.research.google.com/github/markusloecher/DataScience2021/blob/main/TWSM/Class5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Libraries

In [16]:
from google.colab import drive
drive.mount('/content/drive')
TWSM_path = "/content/drive/MyDrive/teaching/TWSM/"

#from TWSM import *
from tensorflow import keras
from tensorflow.keras import layers
import spacy
## Import packages
import pickle
import pandas as pd
from gensim.models import Word2Vec

# run this from a normal command line
#!python -m spacy download en_core_web_md #160MB

#can I download this to a local file instead and load it fom drive?


Mounted at /content/drive


In [ ]:
!pip install whatlies

https://stackoverflow.com/questions/56927602/unable-to-load-the-spacy-model-en-core-web-lg-on-google-colab

Now, *** restart the colab runtime *** !!

## word2vec in spacy

In [ ]:
import spacy
# Load the spacy model that you have installed
nlp = spacy.load('en_core_web_md')

# process a sentence using the model
doc = nlp("The sun is shining brightly today but the moon is not")

# It's that simple - all of the vectors and words are assigned after this point
# Get the vector for 'text':
doc[3].vector

# Get the mean vector for the entire sentence (useful for sentence classification etc.)
doc.vector

In [3]:
#Opposites are not necessarily different
doc = nlp("I loved Narnia but hated Armageddon")

print(doc[1])
print(doc[4])
print(doc[1].similarity(doc[4]))
print(doc[4].similarity(doc[1]))

doc = nlp("The king and the queen are enjoying a sumptious breakfast today")

print(doc[1])
print(doc[4])
print(doc[1].similarity(doc[4]))
print(doc[4].similarity(doc[1]))

loved
hated
0.66889775
0.66889775
king
queen
0.72526103
0.72526103


WhatLies module

In [ ]:
king = nlp.vocab["king"].vector
man = nlp.vocab["man"].vector
queen = nlp.vocab["queen"].vector
woman = nlp.vocab["woman"].vector

#or:
def w2v(w="king"):
  return nlp.vocab[w].vector

king = w2v("king")

In [45]:
from whatlies import EmbeddingSet
from whatlies.language import SpacyLanguage

lang = SpacyLanguage('en_core_web_md')
words = ['cat', 'dog', 'fish', 'kitten', 'man', 'woman', 'king', 'queen', 'doctor', 'nurse']

emb = lang[words]
emb.plot_interactive(x_axis='man', y_axis='woman')

alt.LayerChart(...)

**Vector Algebra**

seems much more difficult than in gensim

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

queries = [w for w in nlp.vocab if w.is_lower and w.prob >= -15]

def cos_sim(a, b):
    return cosine_similarity(a.reshape(1, -1), b.reshape(1, -1))

def most_similar_vec(vec, count=10):
    by_similarity = sorted(queries, key=lambda w: cos_sim(w.vector, vec), reverse=True)
    return [w.orth_ for w in by_similarity[:count]]

vec = nlp('woman').vector + nlp('king').vector - nlp("man").vector
most_similar_vec(vec)

['king',
 'queen',
 'prince',
 'princes',
 'kings',
 'princess',
 'princesses',
 'mermaid',
 'royal',
 'royals']

In [ ]:
queries

## word2vec in gensim

In [4]:
import gensim.downloader as api
#https://github.com/RaRe-Technologies/gensim-data
wv = api.load('glove-wiki-gigaword-100')#128MB
#wv = api.load('glove-wiki-gigaword-50')#65MB


[==================================================] 100.0% 128.1/128.1MB downloaded


In [5]:
pairs = [
    ('car', 'minivan'),   # a minivan is a kind of car
    ('car', 'bicycle'),   # still a wheeled vehicle
    ('car', 'airplane'),  # ok, no wheels, but still a vehicle
    ('car', 'cereal'),    # ... and so on
    ('car', 'communism'),
]
for w1, w2 in pairs:
    print('%r\t%r\t%.2f' % (w1, w2, wv.similarity(w1, w2)))

'car'	'minivan'	0.67
'car'	'bicycle'	0.69
'car'	'airplane'	0.65
'car'	'cereal'	0.12
'car'	'communism'	0.04


## Data

**IMD Movie Reviews**

In [ ]:
from tensorflow.keras.datasets import imdb
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(
    num_words=10000)

#### Tasks

1. Train a word2vec model of dimension $100$ on the IMD data. (considering words appearing in more than 50 documents) Save the model if you like.

2. Compute the embedding for each review (average word2vec)

3. Fit a keras classifier to the embedded reviews. (2 hidden layers of size 40 each) Report/Monitor the accuracy on the test data.

4. Load the bing sentiment dictionary. Compute two separate embeddings for the negative and positive sentiments.

5. Compute the similarity between these two vectors and a few selected reviews. Does it agree with their label?


-------------------------------------------
## Solution